In [1]:
# === CLEAN IMPORTS ===
import numpy as np
import pandas as pd
import re

# Scalers / CV / Models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# ========= 1) LOAD =========
train_df = pd.read_csv("train.csv")
test_df  = pd.read_csv("test.csv")

# Giữ nhãn và tách phần không nhãn
y = train_df["Survived"].astype(int)
train_X = train_df.drop(columns=["Survived"])

# Dùng concat thay cho append (đã deprecated)
full = pd.concat([train_X, test_df], axis=0, ignore_index=True)

# ========= 2) FEATURE ENGINEERING =========
# 2.1 Title từ Name
full["Title"] = full["Name"].str.extract(r" ([A-Za-z]+)\.", expand=True)
# Gom nhóm title hiếm như kernel cũ
title_map = {
    "Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs",
    "Major": "Mr", "Col": "Mr", "Sir": "Mr", "Don": "Mr", "Jonkheer": "Mr", "Capt": "Mr",
    "Lady": "Mrs", "Countess": "Mrs", "Dona": "Mrs"
}
full["Title"] = full["Title"].replace(title_map)
# Chỉ giữ các nhóm chính; cái khác xem như "Other" để median ổn định
major_titles = {"Mr","Mrs","Miss","Master","Dr","Rev"}
full["Title"] = np.where(full["Title"].isin(major_titles), full["Title"], "Other")

# 2.2 Impute Age theo Title (median) – vectorized
# Dùng toàn bộ full để tính median như bạn nói (không dùng nhãn)
age_median_by_title = full.groupby("Title")["Age"].median()
full["Age"] = full["Age"].fillna(full["Title"].map(age_median_by_title))
# fallback nếu còn NaN
full["Age"] = full["Age"].fillna(full["Age"].median())

# 2.3 Family_Size
full["Family_Size"] = full["Parch"].fillna(0) + full["SibSp"].fillna(0)

# 2.4 Family_Survival (dựa theo S.Xu – nhưng tính QUY TẮC trên TRAIN để tránh rò rỉ)
# Chuẩn bị dataframe tạm có Survived chỉ với train
tmp = train_df.copy()
tmp["Last_Name"] = tmp["Name"].str.split(",").str[0]
tmp["Fare"] = tmp["Fare"].fillna(tmp["Fare"].median())

DEFAULT_SURV = 0.5
# Quy tắc suy luận theo (Last_Name, Fare) trong TRAIN
fam_rules = {}

for (lname, fare), grp in tmp.groupby(["Last_Name", "Fare"], dropna=False):
    if len(grp) <= 1:
        continue
    smax = grp["Survived"].max()
    smin = grp["Survived"].min()
    if smax == 1:
        fam_rules[("LF", lname, fare)] = 1.0
    elif smin == 0:
        fam_rules[("LF", lname, fare)] = 0.0

# Quy tắc theo Ticket trong TRAIN
for ticket, grp in tmp.groupby("Ticket", dropna=False):
    if len(grp) <= 1:
        continue
    smax = grp["Survived"].max()
    smin = grp["Survived"].min()
    if smax == 1:
        fam_rules[("T", ticket)] = 1.0
    elif smin == 0:
        fam_rules[("T", ticket)] = 0.0

# Áp quy tắc vào FULL (test không có Survived nên chỉ nhận quy tắc từ train, không rò rỉ)
full["Last_Name"] = full["Name"].str.split(",").str[0]
full["Fare"] = full["Fare"].astype(float)
full["Fare"] = full.groupby("Pclass")["Fare"].transform(lambda s: s.fillna(s.median()))  # điền thiếu trước

def infer_family_survival(row):
    # Ưu tiên quy tắc theo Last_Name + Fare
    key1 = ("LF", row["Last_Name"], row["Fare"])
    if key1 in fam_rules:
        return fam_rules[key1]
    # Sau đó đến Ticket
    key2 = ("T", row["Ticket"])
    if key2 in fam_rules:
        return fam_rules[key2]
    return DEFAULT_SURV

full["Family_Survival"] = full.apply(infer_family_survival, axis=1)

# 2.5 FareBin / AgeBin (ordinal) → mã hoá bằng cat.codes
full["Fare"] = full["Fare"].fillna(full["Fare"].median())
full["FareBin"] = pd.qcut(full["Fare"], 5, duplicates="drop")
full["FareBin_Code"] = full["FareBin"].cat.codes

full["AgeBin"] = pd.qcut(full["Age"], 4, duplicates="drop")
full["AgeBin_Code"] = full["AgeBin"].cat.codes

# 2.6 Map Sex
full["Sex"] = full["Sex"].replace({"male":0, "female":1})

# ========= 3) CLEAN COLUMNS =========
# Theo kernel cũ: drop những cột không dùng để fit
drop_cols = ["Name","PassengerId","SibSp","Parch","Ticket","Cabin","Embarked","Fare","Age","FareBin","AgeBin","Last_Name"]
X_full = full.drop(columns=[c for c in drop_cols if c in full.columns])

# X_full hiện gồm: Pclass, Sex, Family_Size, Family_Survival, FareBin_Code, AgeBin_Code, Title
# Encode Title (ordinal nhỏ gọn) – hoặc OneHot; ở đây ta map đơn giản như kernel cũ
title_order = {"Mr":0, "Mrs":1, "Miss":2, "Master":3, "Dr":4, "Rev":5, "Other":6}
X_full["Title"] = X_full["Title"].map(title_order).astype(int)

# Tách lại train/test theo kích thước
X = X_full.iloc[:len(train_df)].reset_index(drop=True)
X_test = X_full.iloc[len(train_df):].reset_index(drop=True)

# ========= 4) SCALING =========
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

# ========= 5) GRID SEARCH KNN (scoring = ROC_AUC như bạn) =========
param_grid = {
    "n_neighbors": [6,7,8,9,10,11,12,14,16,18,20,22],
    "weights": ["uniform", "distance"],
    "leaf_size": list(range(1, 50, 5)),
    "algorithm": ["auto"],   # có thể thêm 'ball_tree','kd_tree' nếu muốn thử
    "p": [1, 2],             # Manhattan / Euclidean
}

knn = KNeighborsClassifier()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    verbose=1
)
grid.fit(X_scaled, y)

print("Best ROC_AUC:", grid.best_score_)
print("Best Estimator:", grid.best_estimator_)

# ========= 6) TRAIN FULL + PREDICT =========
best_knn = grid.best_estimator_
best_knn.fit(X_scaled, y)
y_pred = best_knn.predict(X_test_scaled).astype(int)

# ========= 7) SUBMISSION =========
sub = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": y_pred
})
sub.to_csv("submission3.csv", index=False)
print("Saved to /submission.csv")
#0.79


Fitting 10 folds for each of 480 candidates, totalling 4800 fits


/tmp/ipykernel_69101/2696515404.py:104: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full["Sex"] = full["Sex"].replace({"male":0, "female":1})


Best ROC_AUC: 0.9127594997594997
Best Estimator: KNeighborsClassifier(leaf_size=1, n_neighbors=20, p=1)
Saved to /submission.csv
